In [5]:
import os
import pydicom
import shutil

In [6]:
directory_path = "D:\\CTH_archive\\DICOM\\ALFORD_BARBARA 4024996\\2019-11-30 163115"
matching_files = []

# Iterate through all files in the specified directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    if file_path.lower().endswith('.dcm'):
        try:
            
            ds = pydicom.dcmread(file_path)
            
            series_description = ds.get((0x0008, 0x103E), "")
            
            if 'PN:01' in series_description.value:
                matching_files.append(file_path)
                
        except Exception as e:
            print(f"Error reading {file_path}: {e}")


if matching_files:
    print("Matching DICOM files:")
    for file in matching_files:
        print(file)
else:
    print("No matching DICOM files found.")


Matching DICOM files:
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00001.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00002.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00003.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00004.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00005.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00006.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00007.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00008.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00009.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00010.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00011.dcm
D:\CTH_archive\DICOM\ALFORD_BARBARA 4024996\2019-11-30 163115\IMG-0002-00012.dcm
D:\CTH

In [7]:
destination_directory_path = "D:\CTH_archive\PN1"

# Copy matching files to the destination directory
for file_path in matching_files:
    # Define the destination path for the file
    destination_path = os.path.join(destination_directory_path, os.path.basename(file_path))
    
    # Copy the file to the destination directory
    shutil.copy2(file_path, destination_path)

print(f"Copied {len(matching_files)} files to {destination_directory_path}")

Copied 23 files to D:\CTH_archive\PN1


In [10]:
import subprocess
import os

# Define the source directory containing the DICOM directories
source_directory_path = "D:\CTH_archive\PN1"

# Define the destination directory where the NIfTI files will be saved
destination_directory_path = "D:\CTH_archive\PN1_NFTI"

# Ensure the destination directory exists
if not os.path.exists(destination_directory_path):
    os.makedirs(destination_directory_path)

# Iterate through all subdirectories in the source directory
for subdir in os.listdir(source_directory_path):
    dicom_dir_path = os.path.join(source_directory_path, subdir)
    
    # Check if the path is a directory
    if os.path.isdir(dicom_dir_path):
        # Construct the dcm2niix command
        # -o specifies the output directory
        # -f specifies the filename format; %p will use the protocol name
        command = ["dcm2niix", "-o", destination_directory_path, "-f", "%p", dicom_dir_path]
        
        # Run the dcm2niix command
        subprocess.run(command)

print(f"Conversion to NIfTI format completed. Files are saved in {destination_directory_path}")


c:\Users\nprim\miniconda3\envs\tf\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path_to_your_dicom_folder'

In [12]:
import os
import numpy as np
import pydicom
import nibabel as nib

def convert_dicom_folder_to_nifti(dicom_folder, output_file):
    # List to store the image arrays
    slices = []

    # Sort DICOM files by Instance Number
    dicom_files = [f for f in os.listdir(dicom_folder) if f.endswith('.dcm')]
    dicom_files.sort(key=lambda f: pydicom.dcmread(os.path.join(dicom_folder, f)).InstanceNumber)

    # Read each DICOM file
    for filename in dicom_files:
        filepath = os.path.join(dicom_folder, filename)
        ds = pydicom.dcmread(filepath)

        # Convert pixel data to numpy array and store
        slices.append(ds.pixel_array)

    # Stack slices into a single 3D array
    # Note: You may need to adjust the order depending on the orientation of your DICOM images
    volume = np.stack(slices, axis=-1)

    # Create a NIfTI image (assuming no header info, such as affine transformation)
    nifti_img = nib.Nifti1Image(volume, affine=np.eye(4))

    # Save the NIfTI image
    nib.save(nifti_img, output_file)

# Specify the DICOM folder and the output NIfTI file path
dicom_folder = "D:\CTH_archive\PN1"
output_nifti_file = 'D:\CTH_archive\PN1_NFTI\output.nii'

# Convert the DICOM folder to NIfTI format
convert_dicom_folder_to_nifti(dicom_folder, output_nifti_file)

print(f"Conversion completed. NIfTI file saved as {output_nifti_file}")


Conversion completed. NIfTI file saved as D:\CTH_archive\PN1_NFTI\output.nii
